DoWhy'da birkaç istisna dışında nedensel bir görevi gerçekleştirmenin ilk adımı nedensel ilişkileri nedensel bir grafik biçiminde modellemektir. Nedensel bir grafik, bir sistem veya sorun alanında mevcut olan nedensel ilişkileri veya “neden-sonuç ilişkilerini” modeller. Bu, her nedensel varsayımı açık bir şekilde ortaya koymaya hizmet eder. Örneğin; rakım → sıcaklık, yani daha yüksek rakım daha düşük sıcaklığa neden olur. DoWhy'da nedensel grafiğin, X→Y kenarının X'in Y'ye neden olduğunu ima ettiği, yönlendirilmiş döngüsel olmayan bir grafik (directed acyclic graph-DAG) olmasını isteriz. İstatistiksel olarak nedensel bir grafik, değişkenler arasındaki koşullu bağımsızlık ilişkilerini kodlar.

Genellikle nedensel grafik, alan bilgisine dayalı olarak oluşturulabilir. Örneğin, dağıtılmış bir mikro hizmet sisteminde, o sistemdeki bağımlılıkların grafiğini yeniden oluşturmak için istek izleme çerçevelerini kullanabiliriz. Buradaki nedensel grafik, bağımlılık grafiğinin tersidir.

![DoWhy](../img/microservice-architecture1.png)

Otel rezervasyonu iptallerine yol açan faktörlerin anlaşılması gibi diğer durumlarda nedensel grafik hakkında bilgi edinmek ve onu oluşturmak için alan uzmanlarına danışabiliriz.

![DoWhy](../img/hotel-booking-cancellations.png)

Nedensel grafiği bilmediğimiz durumlarda verilerden nedensel yapıları öğrenmeye yönelik yöntemler uygulayabiliriz. Verilerden nedensel yapıyı öğrenme bölümü bunun için istatistiksel yöntemleri tanıtmaktadır.

Nedensel grafik, çoğu nedensel görev için en önemli bileşendir çünkü aşağı yönlü analizin doğruluğu buna bağlıdır. Nedensel bir grafiği teşhis etmek ve potansiyel olarak reddetmek için Nedensel Grafiği Reddetmek ve Kullanıcı Tarafından Verilen Yönlendirilmiş Döngüsel Grafiklerin Sahteleştirilmesi örnek not defterine göz atın. Bu testler bir grafiğin geçerli olduğu sonucuna varamaz ancak bir veri kümesi için herhangi bir geçerli grafiğin karşılaması gereken gerekli özellikleri kontrol eder. Sonuç olarak geçersiz grafikleri çürütmek, hataları keşfetmeye yardımcı olmak ve grafiği iyileştirmek için kullanılabilirler.

Nedensel grafiği elde ettiğimizde sonraki adımlar yapmak istediklerimize göre tanımlanır:

* Belirli etki tahmincilerini kullanarak etki tahmini için ihtiyacımız olan tek şey budur. Bir sonraki adım, Belirli Etki Tahmincilerini Kullanarak Etki Tahmininde (ACE için, aracılık etkisi,…) açıklandığı gibi “tanımlama” olacaktır.
* Diğer birçok görev için, Grafiksel Nedensel Modellerin Modellenmesi (GCM'ler) bölümünde açıklandığı gibi her bir düğüme sözde nedensel mekanizmalar atamamız gerekecektir.

# 1.Alan Bilgisi ile Nedensel Grafik Oluşturma

DoWhy'da nedensel grafikler oluşturmak için NetworkX kütüphanesini kullanıyoruz. Aşağıdaki kod parçasında X→Y→Z zinciri oluşturuyoruz:

In [1]:
import networkx as nx
causal_graph = nx.DiGraph([('X', 'Y'), ('Y', 'Z')])

Networkx grafik nesnesi daha sonra doğrudan DoWhy fonksiyonlarına aktarılabilir. Alternatif olarak, networkx grafiğinin GML dize gösterimini kullanarak bir CausalModel örneğini oluşturabilirsiniz.

```python

from dowhy import CausalModel
import networkx as nx
model = CausalModel(
   data=df, # some pandas dataframe
   treatment="v0",
   outcome="y",
   graph="\n".join(nx.generate_gml(causal_graph))
)
```

Nedensel göreve bağlı olarak grafiğin tamamen belirtilmesine gerek olmadığını unutmayın. Örneğin, Etki Tahmini için belirli Etki Tahmincileri (ACE için, aracılık etkisi,…) kullanılarak bazı değişkenler hakkında ön bilgileri (prior knowledge) temsil eden kısmi bir grafik sağlanabilir. DoWhy, değişkenlerin geri kalanını otomatik olarak potansiyel karıştırıcı (confounder) olarak değerlendirir. Veya alternatif olarak, hedef nedensel miktarın tanımlanması için gereken değişkenlerin adları sağlanabilir; örneğin, araçsal değişkenler veya etki tahmin görevi için tedavinin ve sonucun ortak nedenleri.

```python	
model = CausalModel(
   data=df, # some pandas dataframe
   treatment="v0",
   outcome="y",
   common_causes=["w", "z"],
)
```

# 2. Verilerden Nedensel Grafik Yapısını Öğrenme

Nedensel grafiği öğrenmek, yalnızca grafiği alan bilgisine veya diğer bilgi kaynaklarına dayanarak oluşturamadığımız durumlarda gereklidir. Eğer spesifik probleminizde zaten nedensel bir grafik varsa, bu bölümü atlayabilir ve Nedensel Grafiği Çürütmeye geçebilirsiniz.

Çoğu durumda belirli bir veri kümesinin gerçek nedensel yapısı bilinmeyebilir. Veri kümesiyle tutarlı aday nedensel grafikler sağlamak için grafik keşif algoritmalarından yararlanabiliriz. Bu tür grafikler daha sonra bir kullanıcı tarafından kendi alan uzmanlığına veya dünya bilgisine uyacak şekilde incelenebilir, düzenlenebilir ve değiştirilebilir. Bu grafiklerin doğrudan kullanılmaması gerektiğini unutmayın çünkü gözlemsel verilerden grafik keşfi, tamamen parametrik olmayan ortamda kanıtlanabilir şekilde imkansız bir sorundur. Bir veri kümesi göz önüne alındığında, tam olarak aynı ortak dağılıma yol açacak ve dolayısıyla veri kümesine göre ayırt edilemeyecek birden fazla grafik mevcuttur (bu tür grafikler Markov eşdeğerlik sınıfını oluşturur). Sonuç olarak, grafik bulma algoritmaları bir grafiği öğrenmek için belirli varsayımlarda bulunur ve öğrenilen bir grafiğin geçerliliğini garanti etmez.

DoWhy, grafik keşif algoritmalarını uygulamaz ancak öğrenilen grafiğin bir keşif algoritmasından girilmesi için basit bir yol sağlar. Tek kısıtlama, Do Why'ın algoritmanın yönlendirilmiş bir döngüsel olmayan grafik (DAG) çıktısı vermesini beklemesidir. Gelecekte, PyWhy'daki causal-learn ve dodiscover paketleriyle entegrasyon yoluyla nedensel grafiklerin öğrenilmesini doğrudan desteklemeyi umuyoruz.

## 2.1.CDT (Causal Discovery Toolbox) ile Nedensel Keşif

Pandas DataFrame olarak bir veri kümesi verildiğinde, aşağıdaki kod parçası grafiği LiNGAM algoritmasını kullanarak öğrenir ve onu Do Why'a yükler. Algoritma uygulaması, ayrı olarak yüklenmesi gereken Nedensel Keşif Araç Kutusu (CDT) paketindedir.

```python
from cdt.causality.graph import LiNGAM
causal_graph = LiNGAM().predict(dataset)
```

# 3.Nedensel Grafiği Çürütmek

Artık problemimiz için nedensel bir grafik elde ettiğimize göre doğal olarak şu soru ortaya çıkıyor: Nedensel grafik doğru mu? Başka bir deyişle, grafik mevcut verilerle tutarlı mı?

Bu soruyu cevaplamak için her nedensel grafiğin, düğümleri üzerinde bir dizi koşullu bağımsızlık ifadesi gerektirdiği gerçeğini kullanıyoruz. Bu koşullu bağımsızlıklara Yerel Markov Koşulları (local markov conditions-LMC'ler) adı verilir. Veri kümesi grafiğin ima ettiği LMC'lerden herhangi birini karşılamıyorsa grafik geçersizdir.

Yerel Markov koşullarını anlamak için üç düğümlü bir sistem düşünelim: X, Y ve Z. Bu düğümler üç grafik yapısında düzenlenebilir:

1. Zincir (chain), Z→X→Y
2. Çatal (fork), Z←X→Y
3. Çarpıştırıcı (collider), Z→X←Y

İlk iki grafik, X verildiğinde Z ve Y'nin bağımsız olması gerektiğini gerektirir. Üçüncü grafik ise Z ve Y'nin bağımsız olduğunu, ancak X'e koşullandırıldığında bağımlı hale geleceklerini gerektirir. Bu koşulların nasıl türetildiğini anlamak için şu adresi okuyabilirsiniz: d-ayrılma.

Bir aday grafik ve bir veri kümesi verildiğinde, bu koşulları belirli bir grafiği çürütmek için kullanabiliriz. Örneğin, bir kullanıcı bir veri kümesi için Zincir grafiği sağlarsa Z ⫫ Y | X. Değilse, kullanıcı tarafından sağlanan grafiğin veri kümesi için geçerli bir DAG olmadığı sonucuna varabiliriz. Ancak veri kümesinin kısıtlamayı sağlaması, kullanıcı tarafından sağlanan grafiğin geçerli olduğu anlamına gelmez. Gerçek grafiğin aynı koşullu bağımsızlığı da ifade eden Çatal grafiği olması muhtemeldir. Bu nedenle test bazı yanlış grafikleri çürütebilir ancak genel olarak bir veri kümesi için doğru grafiği benzersiz şekilde belirleyemez. Örneğimize devam edersek, kullanıcı Zincir grafiğinin geçersiz olduğunu tespit ederse grafiği düzenleyebilir ve bir Çarpıştırıcı grafiği önerebilir. Çürütme testindeki koşullu bağımsızlık kısıtlaması artık grafiği geçersiz kılamayacak ve alt analize geçilebilecektir.

Daha büyük grafikler için koşullu bağımsızlık testlerinin sayısı büyük bir liste olabilir. DoWhy bu testleri otomatik olarak sıralayabilir ve çalıştırabilir. Ayrıca Kullanıcı Tarafından Verilen Yönlendirilmiş Döngüsel Olmayan Grafiklerin Yanlışlanması (falsification) adlı örnek not defterine de bakın. Daha sonra, tek bir bağımsızlık kısıtının nasıl test edileceğini ve ardından tam grafiğin geçerliliğinin nasıl test edileceğini göstereceğiz.

## 3.1.Bağımsızlık Testleri

Üç değişkenden oluşan aşağıdaki veri kümesini ele alalım:

In [2]:
import numpy as np 
import pandas as pd

X = np.random.normal(loc=0, scale=1, size=1000)
Y = 2 * X + np.random.normal(loc=0, scale=1, size=1000)
Z = 3 * Y + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))

Nedensel grafiğin $X \rightarrow Y \rightarrow Z$ olduğunu varsayalım. Bu, $Y$ verildiğinde $X$ ve $Z$'nin bağımsız olduğu anlamına gelir. Bu koşulu kernel bağımsızlık ölçümü kullanarak test etmek için:

In [3]:
import dowhy.gcm as gcm
# Null hypothesis: x is independent of y given z
p_value = gcm.independence_test(X, Z, conditioned_on=Y, method='kernel')
p_value

Örneğin 0,05'lik bir eşik tanımlarsak, bağımsızlığı ancak p değeri bunun altına düşerse reddederiz. Kesin konuşmak gerekirse yalnızca bağımsızlığı reddedebileceğimizi, ancak p değeri eşiğin üzerindeyse bunu kabul edemeyeceğimizi unutmayın. Ancak pratikte yukarıdaki sonuç yine de değişkenlerin koşullu bağımsız olduğuna dair bir kanıt olarak değerlendirilebilir. Bu aynı zamanda, $X \rightarrow Y \rightarrow Z$ grafiği kullanarak ürettiğimiz veriden beklenecek bir sonuçtur.

Doğru grafik yerine veri kümesi için şu grafiği sağladığımızı düşünelim: $X \rightarrow Y \leftarrow Z$. Grafiğe göre, $Y$ bir çarpıştırıcıdır ve $X$ ve $Z$'nin bağımsız olması gerekir. Bu koşulu test etmek için:

In [4]:
# Null hypothesis: x is independent of y
p_value = gcm.independence_test(X, Z, method='kernel')
p_value

Yine 0,05 gibi bir eşik değeri tanımlayabiliriz ancak bu kez p değeri bu eşiğin açıkça altındadır ve bağımsızlığın sıfır hipotezini açıkça reddedebiliriz. Bu, veri kümesinin $X \rightarrow Y \leftarrow Z$ grafiğini desteklemediği anlamına gelir. $X$ ve $Z$'nin bağımsız olmadığına dair kanıt olarak değerlendirilebilir.

## 3.2.Grafik Çürütmeleri

Tam bir grafiğin geçerliliğini test etmek için, yerel Markov koşulları (LMC'ler) olarak bilinen, grafiğin ima ettiği koşullu bağımsızlık kısıtlamalarının her birini test etmemiz gerekir. Bir grafiğin geçerliliğini test etmek için şunu yazabiliriz:

```python
from dowhy.gcm.falsify import falsify_graph
# causal_graph is a networkx digraph
result = falsify_graph(causal_graph, data, show_progress_bar=False)
print(result)
```

falsify_graph'ın sonuçları iki testin çıktısını gösterir. İlki, grafiğin ima ettiği LMC'lerin veriler tarafından karşılanıp karşılanmadığını ölçer. Verilen grafik tarafından ihlal edilen LMC'lerin sayısını rastgele grafikler tarafından ihlal edilen LMC'lerin sayısıyla karşılaştırır. 0,05 anlamlılık değeri için, eğer verilen grafikteki LMC ihlallerinin sayısı %5 en iyi rastgele grafiklerden düşükse, o zaman grafiği reddetmeyiz. İkinci test (tPa), grafiğin yanlışlanabilir olup olmadığını kontrol eder. Yani, verilen grafiğin doğru olduğunu varsayarsak, başka kaç grafik aynı sayıda LMC ihlalini paylaşıyor? Grafiğin doğru olduğu varsayıldığından, doğru LMC'ler grafiğin ima ettiği LMC'lerdir ve dolayısıyla ihlallerin referans sayısı sıfırdır. 0,05 anlamlılık değeri için, eğer rastgele grafiklerin %5'inden azında sıfır LMC ihlali varsa, bu, grafiğin ima ettiği LMC'lerin grafiği yanlışlayabileceğini (veya çürütebileceğini) gösterir.

Eğer bir grafiğin yanlışlanabilir olup olmadığını öğrenmek istiyorsak doğrudan sonuç nesnesini sorgulayabiliriz.

```python
print(f"Graph is falsifiable: {result.falsifiable}, Graph is falsified: {result.falsified}")
```